In [41]:
# load libraries
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine, text
import zipfile

## Setup

In [42]:
# Welcher Zip?
source = "delfi" # where is the feed file? delfi or gtfs.de?
zipname = '20220425_fahrplaene_gesamtdeutschland_gtfs' # name of GTFS zipfile

# define paths
workingdir = "../../data/" 
#storagedir = "smb://192.168.90.30/allmende%20verkehr/4%20Projekte/2%20Projekte%20Mobilitätswende/ÖV-Deutschlandkarte%20(Gap-Map)/Berechnungen/raw/gtfs/"

# constructed paths
rawdir = workingdir + "raw/" # where is all the data?
gtfsdir = rawdir + source + "/" # where zip-file is located
outdir = workingdir + "interim/" # where do outputfiles go?
zippath = gtfsdir + zipname + ".zip"

# set up zip file as default for functions
zf = zipfile.ZipFile(zippath) # this is the raw stuff

In [43]:
# choose file-based output connection
outpath = '{0}{1}.db'.format(outdir,zipname)
# set up DB connection
dbout = create_engine('sqlite:///' + outpath)

In [44]:
# file for logging
logfile = outdir + zipname + ".log"
with open(logfile, 'a') as f:
    f.write('## Eckdaten\n')

# Count service_ids

In [45]:
def interveningWeekdays(start, end, inclusive=True, weekdays=[0, 1, 2, 3, 4]):
    # a useful function from Stackoverflow, to count particular weekdays in date range
    if isinstance(start, dt.datetime):
        start = start.date()               # make a date from a datetime

    if isinstance(end, dt.datetime):
        end = end.date()                   # make a date from a datetime

    if end < start:
        # you can opt to return 0 or swap the dates around instead
        # raise ValueError("start date must be before end date")
        end, start = start, end

    if inclusive:
        end += dt.timedelta(days=1)  # correct for inclusivity

    try:
        # collapse duplicate weekdays
        weekdays = {weekday % 7 for weekday in weekdays}
    except TypeError:
        weekdays = [weekdays % 7]
        
    ref = dt.date.today()                    # choose a reference date
    ref -= dt.timedelta(days=ref.weekday())  # and normalize its weekday

    return sum((ref_plus - start).days // 7 - (ref_plus - end).days // 7
               for ref_plus in
               (ref + dt.timedelta(days=weekday) for weekday in weekdays))

def countDaysInIntervalHelper(calendarrow):
    # function to find number of days of service operation based on calendars.txt-entry
    servicepattern = calendarrow.loc["monday":"sunday"].to_numpy()
    servicedays = servicepattern.nonzero()[0].tolist()

    startdate = dt.datetime.strptime(str(int(calendarrow.get("start_date"))),"%Y%m%d")
    enddate = dt.datetime.strptime(str(int(calendarrow.get("end_date"))),"%Y%m%d")
    return(interveningWeekdays(startdate, enddate, weekdays = servicedays))

### Helper function to compare dates
def isInIntervalHelper(n, interval):
    '''works only on ARRAY-like n'''
    return(np.where((n <= max(interval)) & (n >= min(interval)), True, False))

In [46]:
# function to add frequencies...
def addCountToCalendar(calendar_df, calendar_dates_df):
    # enriches stop_times DataFrame with information about how often in the feed
    # period each stop is made
    

    print("Getting number of service days for each service")
    # use service_id to find service...
    calendar_df["days_count"] = calendar_df.apply(countDaysInIntervalHelper, axis=1)    

    print("\t...aggregating calendar")
    calendar_df = calendar_dates_df.groupby(["service_id", "exception_type"], as_index=False
                              ).count(
                            ).pivot(index = "service_id", columns = "exception_type", values = "date"
                            ).reset_index(
                            ).merge(calendar_df, on="service_id", how="right"
                            )[['service_id', 1, 2, 'monday',
                                  'tuesday',  'wednesday',   'thursday',     'friday',   'saturday',
                                  'sunday', 'start_date',   'end_date', 'days_count']]
    
    print("\t...calculating total in calendar")
    calendar_df.days_count= (calendar_df.days_count + calendar_df[1].fillna(0) - calendar_df[2].fillna(0)
                            )
    
    return(calendar_df)

In [47]:
def feedDays(calendar_df, calendar_dates_df):
    ''' Enriches counted dataframe with average daily count for each stop,
    using the feed's calendar information to infer the number of days
    '''
    # calculate
    startdate =  min(pd.to_datetime(calendar_df.start_date,format="%Y%m%d"))
    enddate = max(pd.to_datetime(calendar_df.end_date,format="%Y%m%d"))
    excdates = pd.to_datetime(calendar_dates_df.date,format="%Y%m%d")

    firstdate = min(startdate, min(excdates))
    lastdate = max(enddate, max(excdates))

    ndays = (lastdate - firstdate).days
    with open(logfile, 'a') as f:
        f.write('First date:\t{}\n'.format(firstdate))
        f.write('Last date:\t{}\n'.format(lastdate))
        f.write('Total days:\t{}\n\n'.format(ndays))
    print('First date:\t',firstdate)
    print('Last date:\t',lastdate)
    print('Total days:\t',ndays)
    return(ndays)

In [48]:
calendar_df = pd.read_csv(zf.open("calendar.txt"))
calendar_dates_df = pd.read_csv(zf.open("calendar_dates.txt"))

In [49]:
calendar_df = addCountToCalendar(calendar_df, calendar_dates_df)

Getting number of service days for each service
	...aggregating calendar
	...calculating total in calendar


In [50]:
ndays = feedDays(calendar_df, calendar_dates_df) # total number of days in feed period

First date:	 2022-04-11 00:00:00
Last date:	 2022-12-10 00:00:00
Total days:	 243


# Pick out routes

This adds a FZ and FB-flag to the routes file, based on previous filtering of FV/NV-routes

In [51]:
fz_routes = pd.read_csv(outdir + zipname + '_fz-routes.csv').route_id
fb_routes = pd.read_csv(outdir + zipname + '_fb-routes.csv').route_id

routes_df = pd.read_csv(zf.open("routes.txt"))

routes_df['fz'] = routes_df.route_id.isin(fz_routes)
routes_df['fb'] = routes_df.route_id.isin(fb_routes)

# Get things into database

## calendar

In [52]:
# put enriched calendar into database
calendar_df.to_sql("calendar", 'sqlite:///' + outpath,
          if_exists = 'replace')

23285

## routes

In [53]:
routes_df.to_sql("routes", 'sqlite:///' + outpath,
          if_exists = 'replace')

26191

## trips, stops, et al.--everything that goes straight into DB

Transfer gtfs-files into database in chunks

In [54]:
%%time
start = dt.datetime.now()
chunksize = 200000

ziptables = ['stops','trips', 'stop_times']
    
    
for table_name in ziptables:
    print(table_name)

    j=0
    for df in pd.read_csv(zf.open(table_name + ".txt"),
                          chunksize=chunksize, iterator=True, encoding='utf-8',
                           dtype={'Unnamed: 0': 'float64',
                           'drop_off_type': 'object',
                           'pickup_type': 'object',
                           'stop_sequence': 'object',
                           'trip_id': 'object',
                           'stop_headsign': 'object'}
                         ):
        j+=1
        if j%10==0: # track progress visibly
            print('\t{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

        if j==1:
            df.to_sql(table_name, dbout, if_exists='replace')
        else:
            df.to_sql(table_name, dbout, if_exists='append')

stops


<timed exec>:11: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.


trips


<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.


	32 seconds: completed 2000000 rows
stop_times
	53 seconds: completed 2000000 rows
	76 seconds: completed 4000000 rows
	98 seconds: completed 6000000 rows
	120 seconds: completed 8000000 rows
	142 seconds: completed 10000000 rows
	164 seconds: completed 12000000 rows
	186 seconds: completed 14000000 rows


<timed exec>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


	208 seconds: completed 16000000 rows
	231 seconds: completed 18000000 rows
	253 seconds: completed 20000000 rows
	275 seconds: completed 22000000 rows
	298 seconds: completed 24000000 rows
	321 seconds: completed 26000000 rows
	343 seconds: completed 28000000 rows
	366 seconds: completed 30000000 rows
	388 seconds: completed 32000000 rows
	412 seconds: completed 34000000 rows
	434 seconds: completed 36000000 rows


<timed exec>:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


	456 seconds: completed 38000000 rows
CPU times: user 6min 58s, sys: 6.56 s, total: 7min 4s
Wall time: 7min 43s


# Database Merging

### recovery if crashed on db create

* run setup (without logging file)
* get feed days

Count stop_times per stop using SQL (to keep large files out of working memory)

This is where the filtering by route happens--therefore two different queries, one for each route subset

Zug:

In [55]:
%%time
print(dt.datetime.now())

fz_out_df = pd.read_sql_query(
    'SELECT n_stops.stop_id, n, stop_name, parent_station, stop_lat, stop_lon, location_type '
    'FROM ('
        'SELECT stop_id, SUM(days_count) AS n '
        'FROM ('
            'SELECT routes.route_short_name, routes.route_id, trips.service_id, trips.trip_headsign, trips.direction_id, trips.trip_id '
            'FROM routes '
            'LEFT JOIN trips ON routes.route_id = trips.route_id '
            'WHERE routes.fz IS 1 '
        ') AS trips_fz '
        'LEFT JOIN stop_times ON trips_fz.trip_id = stop_times.trip_id '
        'LEFT JOIN calendar ON trips_fz.service_id = calendar.service_id '
        'GROUP BY stop_id '
    ') AS n_stops '
    'JOIN stops ON n_stops.stop_id = stops.stop_id',
    dbout
)

2022-06-07 17:02:15.882634
CPU times: user 51.6 s, sys: 12.4 s, total: 1min 3s
Wall time: 1min 10s


Bus:

In [56]:
fz_out_df["n_day"] = fz_out_df.n/ndays # the count per day, for comparing different length feeds
fz_out_df.to_csv(outdir + zipname + ".fz.nstops.csv")


In [57]:
%%time
print(dt.datetime.now())

fb_out_df = pd.read_sql_query(
    'SELECT n_stops.stop_id, n, stop_name, parent_station, stop_lat, stop_lon, location_type '
    'FROM ('
        'SELECT stop_id, SUM(days_count) AS n '
        'FROM ('
            'SELECT routes.route_short_name, routes.route_id, trips.service_id, trips.trip_headsign, trips.direction_id, trips.trip_id '
            'FROM routes '
            'LEFT JOIN trips ON routes.route_id = trips.route_id '
            'WHERE routes.fb IS 1 '
        ') AS trips_fb '
        'LEFT JOIN stop_times ON trips_fb.trip_id = stop_times.trip_id '
        'LEFT JOIN calendar ON trips_fb.service_id = calendar.service_id '
        'GROUP BY stop_id '
    ') AS n_stops '
    'JOIN stops ON n_stops.stop_id = stops.stop_id',
    dbout
)

2022-06-07 17:03:28.242176
CPU times: user 47.6 s, sys: 7.52 s, total: 55.2 s
Wall time: 1min 1s


In [58]:
fb_out_df["n_day"] = fb_out_df.n/ndays # the count per day, for comparing different length feeds
fb_out_df.to_csv(outdir + zipname + ".fb.nstops.csv")

Nahverkehr:

In [59]:
%%time
print(dt.datetime.now())

nv_out_df = pd.read_sql_query(
    'SELECT n_stops.stop_id, n, stop_name, parent_station, stop_lat, stop_lon, location_type '
    'FROM ('
        'SELECT stop_id, SUM(days_count) AS n '
        'FROM ('
            'SELECT routes.route_short_name, routes.route_id, trips.service_id, trips.trip_headsign, trips.direction_id, trips.trip_id '
            'FROM routes '
            'LEFT JOIN trips ON routes.route_id = trips.route_id '
            'WHERE routes.fz IS 0 AND routes.fb IS 0'
        ') AS trips_nv '
        'LEFT JOIN stop_times ON trips_nv.trip_id = stop_times.trip_id '
        'LEFT JOIN calendar ON trips_nv.service_id = calendar.service_id '
        'GROUP BY stop_id '
    ') AS n_stops '
    'JOIN stops ON n_stops.stop_id = stops.stop_id',
    dbout
)

2022-06-07 17:04:29.779972
CPU times: user 1min 32s, sys: 12.1 s, total: 1min 44s
Wall time: 1min 51s


In [60]:
nv_out_df["n_day"] = nv_out_df.n/ndays # the count per day, for comparing different length feeds
nv_out_df.to_csv(outdir + zipname + ".nv.nstops.csv")

# Wrap up and write out